
# **Pengujian Baseline Tanpa Optimasi**

Pengujian baseline dilakukan untuk mengevaluasi performa awal Fuzzy Inference System (FIS) sebelum parameter Membership Function (MF) dioptimasi menggunakan GA dan PSO. Pada tahap ini, sistem menggunakan MF default yang disusun secara netral berdasarkan domain probabilitas ([0, 1]) untuk input, dan domain indeks ([0, 5]) untuk output tone chatbot.
Baseline diperlukan sebagai titik acuan untuk melihat seberapa besar peningkatan performa yang diberikan oleh algoritma optimasi di tahap berikutnya.

---

## **Konfigurasi FIS pada Baseline**

### **Membership Function untuk Input (Emosi)**

Keenam emosi (sadness, joy, love, anger, fear, surprise) dipetakan ke tiga MF sederhana:

* **Low**  : ([0.0, 0.0, 0.5])
* **Medium** : ([0.0, 0.5, 1.0])
* **High** : ([0.5, 1.0, 1.0])

Ini merepresentasikan intensitas probabilitas 0–1 dari model Roberta.

---

## **Membership Function untuk Output (Tone Chatbot)**

Berbeda dari input, output FIS tidak berada pada domain probabilitas. Output FIS berupa **nilai kontinu 0–5**, yang setelah dibulatkan menjadi salah satu dari 6 kelas tone:

1. Empathetic
2. Friendly
3. Warm
4. Calming
5. Supportive
6. Informative

Karena itu baseline FIS menggunakan **6 membership function output**—satu untuk setiap tone—yang dibentuk menggunakan triangular MF pada domain:

```
[0.0, 5.0]
```

### **Struktur MF Output pada Baseline**

Untuk tone ke-i (0–5), triangular MF disusun dengan pola:

* kelas 0 → ([0, 0, 1])
* kelas 1 → ([0, 1, 2])
* kelas 2 → ([1, 2, 3])
* kelas 3 → ([2, 3, 4])
* kelas 4 → ([3, 4, 5])
* kelas 5 → ([4, 5, 5])

Secara formal:

[
\text{MF}_i = \text{trimf}(x; [i-1,; i,; i+1])
]

dengan penyesuaian pada kelas pertama dan terakhir agar tetap berada dalam domain ([0,5]).

### **Alasan Pemilihan Parameter MF Output**

1. **Output adalah kategori diskrit (6 kelas), tetapi FIS harus bekerja dalam domain kontinu.**
   Karena FIS harus mengambil keputusan berdasarkan pemetaan fuzzy, setiap kelas tone direpresentasikan sebagai “puncak MF” pada satu titik indeks (0–5).

2. **Triangular MF memberikan transisi halus antar kelas.**
   Misalnya, antara *friendly* (1) dan *warm* (2) terdapat area overlap yang wajar secara semantik.

3. **Penempatan MF yang teratur menjaga stabilitas rule base.**
   Baseline perlu stabil agar ketika GA/PSO melakukan tuning, perubahan parameter tidak memecahkan permukaan inferensi fuzzy.

4. **Simetris dan mudah dioptimasi.**
   Format ([i-1, i, i+1]) membuat ruang pencarian GA/PSO terstruktur, tidak acak, dan mudah dieksplorasi.

5. **Memisahkan domain input (0–1) dan output (0–5).**
   Input berasal dari probabilitas, sedangkan output adalah indeks tone chatbot.
   Karena itu parameternya harus berbeda domain agar representasi fuzzy tetap logis.

Dengan konfigurasi ini, baseline FIS memiliki struktur MF output yang:

* logis secara semantik,
* stabil untuk inference manual,
* dan menjadi starting point yang baik sebelum optimisasi.

---

## **Metrik Evaluasi**

Pengujian baseline dilakukan menggunakan empat metrik utama:

### **1. Akurasi (Accuracy)**

Mengukur persentase prediksi tone chatbot yang sesuai dengan label target.

### **2. F1-Score (Weighted)**

Digunakan karena distribusi data antar kelas tone tidak seimbang.

### **3. Waktu Eksekusi (Execution Time)**

Mengukur durasi inference seluruh data uji menggunakan FIS baseline.

### **4. Stabilitas (Standard Deviation)**

Tidak dihitung pada baseline, karena baseline hanya dijalankan satu kali.
Stabilitas baru relevan pada eksperimen GA dan PSO yang bersifat stokastik.

---



In [3]:
import os
os.chdir("D:/AI_Journey/Computational_Intelligence/fp-kk")
print("Current directory:", os.getcwd())


Current directory: D:\AI_Journey\Computational_Intelligence\fp-kk


In [ ]:
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score

# Load dataset
X_test = np.load("data/processed/X_test.npy")
y_test = np.load("data/processed/y_test.npy")

# Import FIS model baseline
from src.fis.fis_model import FISChatbot

# Inisialisasi FIS baseline (tanpa optimasi)
fis = FISChatbot()

# ------------------------
# Baseline Inference
# ------------------------

start_time = time.time()

y_pred = fis.predict_batch(X_test)

exec_time = time.time() - start_time


# ------------------------
# Metrik Evaluasi
# ------------------------

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")

print("=== Baseline Evaluation ===")
print(f"Akurasi            : {accuracy:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")
print(f"Waktu Eksekusi     : {exec_time:.4f} detik")
print("Stabilitas         : Tidak dihitung (baseline only)")


=== Baseline Evaluation ===
Akurasi            : 0.3566
F1-Score (weighted): 0.2878
Waktu Eksekusi     : 11.1950 detik
Stabilitas         : Tidak dihitung (baseline only)
